# Explore here

In [1]:
# Your code here
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Lasso


In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv", sep = ",")
df.head()


,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,12.749771,...,3644,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3
1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,11.564429,...,14692,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4
2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,12.865239,...,2373,19.7,18.6,20.6,3870,4.5,4.2,4.8,887,6
3,1007,22400,2456,10.964286,2596,11.589286,3029,13.522321,3113,13.897321,...,1789,14.1,13.2,14.9,2511,3.3,3.1,3.6,595,2
4,1009,57840,7095,12.266598,7570,13.087828,6742,11.656293,6884,11.901798,...,4661,13.5,12.6,14.5,6017,3.4,3.2,3.7,1507,2


In [5]:
duplicate_rows = df[df.duplicated()]
if duplicate_rows.empty:
    print("No duplicate values found.")
else:
    print("Duplicate values found.")



No duplicate values found.


In [6]:
data_types = df.dtypes
numeric_columns = [c for c in list(data_types[data_types != "object"].index) if c != "Heart disease_number"]

scaler = StandardScaler()
norm_features = scaler.fit_transform(df[numeric_columns])

df_scal = pd.DataFrame(norm_features, index = df.index, columns = numeric_columns)
df_scal["Heart disease_number"] = df["Heart disease_number"]
df_scal.head()


,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code,Heart disease_number
0,-1.940874,-0.145679,-0.142421,0.158006,-0.135556,0.573496,-0.153144,0.027610,-0.139384,0.588469,...,-0.063696,-0.071720,-0.089834,-0.129902,-0.609615,-0.582796,-0.669652,-0.147523,-1.082865,3345
1,-1.940742,0.341296,0.287476,-0.242861,0.320383,-0.193107,0.183774,-0.469965,0.230620,-0.110300,...,-0.394103,-0.414900,-0.337677,0.376251,-0.433549,-0.393279,-0.343373,0.389791,-0.420704,13414
2,-1.940610,-0.237785,-0.239429,-0.419441,-0.246181,-0.439718,-0.225971,0.272104,-0.218759,0.656538,...,2.432709,2.483064,2.317776,-0.183415,1.855312,1.880929,1.777443,-0.204321,0.903618,2159
3,-1.940478,-0.245223,-0.246032,-0.426966,-0.254791,-0.609076,-0.230792,0.396168,-0.220555,1.264959,...,0.376846,0.423984,0.299632,-0.229096,-0.257483,-0.203761,-0.180233,-0.242100,-1.745026,1533
4,-1.940346,-0.138966,-0.135053,0.186249,-0.137140,0.216679,-0.155888,-0.200808,-0.143570,0.088582,...,0.156575,0.195197,0.158008,-0.111247,-0.081417,-0.014244,-0.017093,-0.124105,-1.745026,4101


In [9]:
X = df_scal.drop(columns=["Heart disease_number"])
y = df_scal["Heart disease_number"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
train_indices = list(X_train.index)
test_indices = list(X_test.index)

k = int(len(X_train.columns) * 0.2)
selection_model = SelectKBest(score_func = f_regression, k = k)
selection_model.fit(X_train, y_train)
ix = selection_model.get_support()

X_train_sel = pd.DataFrame(selection_model.transform(X_train), columns = X_train.columns.values[ix])
X_test_sel = pd.DataFrame(selection_model.transform(X_test), columns = X_test.columns.values[ix])


In [10]:
X_train_sel["Heart disease_number"] = list(y_train)
X_test_sel["Heart disease_number"] = list(y_test)

X_train_sel.to_csv("../data/processed/train.csv", index = False)
X_test_sel.to_csv("../data/processed/test.csv", index = False)


In [11]:
model = LogisticRegression()
model.fit(X_train, y_train)


/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [14]:
model_intercept=model.intercept_
Model_C=model.coef_

print("intercept: " + str(model_intercept))
print("Coefficients: " + str(Model_C))
      

intercept: [-0.37378091 -0.87756298 -0.15249069 ... -1.57861187 -1.57184268
 -1.34247489]
Coefficients: [[ 0.07415864 -0.00779112 -0.00978955 ... -0.03154737 -0.00749716
   0.08156152]
 [-0.00435315 -0.00736242 -0.00675498 ...  0.0692093  -0.0077426
  -0.01031092]
 [ 0.18706976 -0.01287968 -0.0135669  ... -0.006242   -0.01353368
   0.09648147]
 ...
 [-0.02810602  0.14617765  0.14166211 ...  0.01612521  0.15494159
  -0.01814839]
 [-0.01428053  0.15292522  0.14227684 ...  0.01850523  0.16560792
  -0.0120456 ]
 [ 0.00169325  0.15676473  0.14882542 ...  0.01981825  0.16669236
   0.00504114]]


In [15]:
y_pred = model.predict(X_test)
y_pred


array([  912, 14180,  2274,  1919, 14180,   587,   382,   881,  1174,
         368,  1231,  1478,  2969,  1692,  6377,  1017, 47200,  1431,
        1205,   403,  1460,  2153,  1565,   739, 95392,  2656,  1255,
        1977,   369,   317,  1838,  2959,   947,  1022,  4358,  1581,
        1980,  1970,  1293,  2567,  1413, 14102,   159,   221,  2341,
        1014,   549,  2562,  1301,  1406, 19213,   510,   369,   217,
        2699,  1413, 40685,   845,  1794,  1596, 10150,   275,  1010,
        6567,   809,  4427,   369,  1933,   643, 10912,  4301,  1072,
        5496,   559,  1980,  4301, 17184,   372,   328,  1409,  1437,
        7128, 12292,   407,   705,  5584, 32828,   621,  3802,  8250,
        1010,   227,   698,   352,   596,  1686,  2282,   724, 10622,
         124, 43089, 10979,  1696,    61,   299,  2969,   555,  3880,
        6652,  9325,   374,  5711, 14980,  7643,   577, 16376,  2254,
        1255,  5517,  2598,   690,  1165,  2598,  3643,  1387,  1738,
        2302,  3851,

In [16]:
MSE= mean_squared_error(y_test, y_pred)
Score= r2_score(y_test, y_pred)

print("MSE: " + str(MSE))
print("R2 Score: " + str(Score))



MSE: 12401906.885350319
R2 Score: 0.8583051779609306


In [17]:
alpha = 1.0
lasso_model = Lasso(alpha = alpha)

lasso_model.fit(X_train, y_train)

score = lasso_model.score(X_test, y_test)
print("Coefficients:", lasso_model.coef_)
print("R2 score:", score)

Coefficients: [-0.00000000e+00  6.47107873e+03  3.51595668e+02 -0.00000000e+00
 -2.03165510e+03  3.35561092e+01 -1.56183109e+02  9.62360018e+01
 -4.19408387e+02 -5.86141855e-02  4.00459426e+03 -1.22746425e+01
  1.89172387e+03  4.67298910e+00 -4.52689402e+02  0.00000000e+00
  1.85824210e+03 -1.15358846e+02  1.50972019e+03 -7.84053457e+01
 -4.27675389e+03 -0.00000000e+00 -1.18784463e+03  4.49499819e+01
 -4.62089319e+01 -4.22870840e+01 -1.20018426e+03  1.11527644e+02
  1.49095266e+02 -3.84347284e+00 -6.05927730e+02 -1.51418439e+01
  1.05422110e+02  3.24901765e+02  6.42508932e+01 -3.50610062e+00
  0.00000000e+00 -1.56052289e+01  3.72511978e+01 -4.88939937e+01
 -0.00000000e+00  3.77714792e+02  1.59545015e+03 -5.22943884e+02
  5.53366580e+02  6.53826196e+01 -2.48638782e+00  7.52719329e+01
 -0.00000000e+00  2.84057950e+03 -1.11669836e+02  0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -1.62526500e+03 -0.00000000e+00 -1.73891167e+03  5.95312157e+01
 -0.0000000

/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.672e+08, tolerance: 7.097e+07
  model = cd_fast.enet_coordinate_descent(
